## Package Imports ##

In [1]:
import os   # Directory Package
import warnings

#Importing Scientific computing package for python. Library that provides a multidimential array
# object, various derived objects (such as masked arrays and matrices), and an assortment of 
# routines for fast operations on arrays, including mathematical, logical, shape manipulation, sorting,
# selecting, I/O, discrete Fourier transforms, basic linear algebra, basic statistical operations, random
# simulation and much more
import numpy as np

from numpy import genfromtxt

#Importing Pythons Open Source Data Analysis and manipulation library
import pandas as pd

import matplotlib.pylab as plt
from matplotlib import style

import seaborn as sns

import tensorflow as tf

import kaggle

import random as rd
from random import shuffle

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, BatchNormalization
from keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from keras.utils import to_categorical

#Importing Pythons Open Computer Vision Library that is designed to solve computer vision problems.
import cv2

from PIL import Image
from tqdm import tqdm

import pathlib  #Object-Oriented Filesystem paths functionality
                #This module offers classes representing filesystem paths.

In [ ]:
#kaggle.api.dataset_list()


## Download Dataset ##

Download the Kaggle [Fruits 360](https://www.kaggle.com/datasets/moltean/fruits) dataset

In [3]:
#!kaggle datasets download moltean/fruits
from zipfile import ZipFile

with ZipFile("fruits.zip", mode="r") as zip_file:
    #zip_file.printdir()
    for file in zip_file.namelist():
        if file.startswith('fruits-360_dataset/fruits-360'):
            zip_file.extract(file, path='.')

zip_file.close()

In [ ]:
IMG_SIZE = 100
X = []    # Data
Z = []    # Classification

def assign_label(img, fruit_type):
    return fruit_type

def make_train_data(fruit_type, dir): 
    for img in tqdm(os.listdir(dir)):       #tqdm shows a status bar in the jupyter environment
        label = assign_label(img, fruit_type)
        path = os.path.join(dir, img)
        try:
            img = cv2.imread(path, cv2.IMREAD_COLOR)
            #img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(np.array(img))
            Z.append(str(label))
        except cv2.error:
            print("image read and resize failed")
            

## Set up Directory Structure ##

In [ ]:
root_directory = pathlib.Path(r"fruits-360\data-set")

#Iterating down the subtree of root directory and Creating a list of all fruit class Directories 
sub_directory_paths = list(root_directory.glob('*'))   

#for file_path in directory_paths:
#  print(file_path)

#Checking The number of directories in the list
#print(len(directory_paths))  

## Create Training Data ##
Iterate over subdirectory paths and pull each subpath out to get the basename which is used to create the class name in the first parameter (os.path.basename(subpaths)) and use that subpaths path to include the location of the image

In [ ]:
for subpaths in sub_directory_paths:
    make_train_data(os.path.basename(subpaths), subpaths)

## Output data for Eye Test ##

In [ ]:
fig, ax = plt.subplots(5, 2)
fig.set_size_inches(15, 15)
for i in range(5):
    for j in range(2):
        l = rd.randint(0, len(Z))
        ax[i,j].imshow(cv2.cvtColor(X[l], cv2.COLOR_BGR2RGB))
        ax[i,j].set_title('Fruit: ' + Z[l])
        
plt.tight_layout()

In [ ]:
le = LabelEncoder()
Y = le.fit_transform(Z)
Y = to_categorical(Y, 131)
X = np.array(X)
X = X/255

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [ ]:
np.random.seed(42)
rd.seed(42)
tf.random.set_seed(42)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size= (5,5), padding = 'Same', activation='relu', input_shape = (100, 100, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size= (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=96, kernel_size=(3,3), padding= 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(131, activation='softmax'))

In [ ]:
batch_size = 128
epochs = 5

from keras.callbacks import ReduceLROnPlateau
red_lr = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.1)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zoom_range=0.1,
    width_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
History = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                              epochs=epochs, validation_data=(x_test, y_test), 
                              verbose=1, steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()